<a href="https://colab.research.google.com/github/Amitsharma405/Amit_projects/blob/main/files_%26_exceptional_handling_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ques.1 Discuss the scenarios where multithreading is preferable to multiprocessing and scenarios where
multiprocessing is a better choice?

When multithreading is preferable
Best for: I/O-bound workloads

These are tasks that spend most of their time waiting, not computing.

Examples

.Network requests (APIs, web scraping)

.File I/O (reading/writing files, logs)

.Database queries

.Waiting on sockets or user input

   Why threading works well here

.Python threads share the same memory

.While one thread is waiting for I/O, another can run

.The GIL (Global Interpreter Lock) is released during I/O operations

   Benefits

.Lower memory overhead than processes

.Faster startup

.Easy data sharing (same memory space)

  Example use cases

.Web servers handling multiple requests

.Downloading many files concurrently

.Chat servers or real-time I/O systems

  When multiprocessing is a better choice
 Best for: CPU-bound workloads

These tasks spend most of their time doing heavy computation.

 Examples

.Image or video processing

.Machine learning training

.Numerical simulations

.Data compression / encryption

  Why multiprocessing works better

.Each process has its own Python interpreter and GIL

.True parallel execution on multiple CPU cores

.Bypasses the GIL entirely

   Benefits

.Full CPU utilization

.Better performance for compute-heavy tasks

.Crashes in one process don’t affect others

   Trade-offs

.Higher memory usage

.Slower startup

.Data must be serialized (pickled) between processes

ques.2 Describe what a process pool is and how it helps in managing multiple processes efficiently?

A process pool is a high-level abstraction for managing a fixed number of worker processes that execute tasks concurrently. Instead of creating and destroying processes for every task, a pool reuses a set of processes, which makes multiprocessing more efficient and easier to manage.



In [ ]:
from multiprocessing import Pool
import os

def square(n):
    return n * n

if __name__ == "__main__":
    with Pool(processes=4) as pool:
        results = pool.map(square, [1, 2, 3, 4, 5])

    print(results)




[1, 4, 9, 16, 25]


In [ ]:
from concurrent.futures import ProcessPoolExecutor

def compute(n):
    return n ** 2

if __name__ == "__main__":
    with ProcessPoolExecutor() as executor:
        results = list(executor.map(compute, range(6)))

    print(results)

[0, 1, 4, 9, 16, 25]


ques.3 Explain what multiprocessing is and why it is used in Python programs?

Multiprocessing in Python is a technique for running multiple processes simultaneously, where each process has its own memory space and its own Python interpreter. This allows a program to perform multiple tasks in parallel, rather than sequentially.

 What multiprocessing is

.Each process is an independent execution unit

.Processes do not share memory by default

.Each process runs on a separate CPU core (when available)

.Communication happens via inter-process communication (IPC) like queues or pipes

 Why multiprocessing is used in Python
1. To bypass the Global Interpreter Lock (GIL)

The GIL allows only one thread at a time to execute Python bytecode per process.
This means multithreading does not provide true parallelism for CPU-bound tasks.

 Multiprocessing:

.Gives each process its own GIL

.Enables true parallel execution on multiple CPU cores

2. To speed up CPU-bound tasks

Multiprocessing is ideal for workloads that require heavy computation, such as:

.Image and video processing

.Scientific simulations

.Machine learning training

.Data analysis and numerical computation

In [ ]:
from multiprocessing import Process

def task():
    print("Running in a separate process")

if __name__ == "__main__":
    p = Process(target=task)
    p.start()
    p.join()

Running in a separate process


ques.4 Write a Python program using multithreading where one thread adds numbers to a list, and another
thread removes numbers from the list. Implement a mechanism to avoid race conditions using
threading.Lock?

In [ ]:
import threading
import time
import random

shared_list = []
lock = threading.Lock()

def producer():
    for i in range(10):
        time.sleep(random.uniform(0.1, 0.3))
        with lock:
            shared_list.append(i)
            print(f"Producer added {i} | List: {shared_list}")

def consumer():
    for _ in range(10):
        time.sleep(random.uniform(0.2, 0.4))
        with lock:
            if shared_list:
                value = shared_list.pop(0)
                print(f"Consumer removed {value} | List: {shared_list}")
            else:
                print("Consumer found list empty")

producer_thread = threading.Thread(target=producer)
consumer_thread = threading.Thread(target=consumer)

producer_thread.start()
consumer_thread.start()

producer_thread.join()
consumer_thread.join()

print("Final list:", shared_list)


Producer added 0 | List: [0]
Consumer removed 0 | List: []
Producer added 1 | List: [1]
Producer added 2 | List: [1, 2]
Consumer removed 1 | List: [2]
Producer added 3 | List: [2, 3]
Consumer removed 2 | List: [3]
Producer added 4 | List: [3, 4]
Producer added 5 | List: [3, 4, 5]
Consumer removed 3 | List: [4, 5]
Producer added 6 | List: [4, 5, 6]
Producer added 7 | List: [4, 5, 6, 7]
Consumer removed 4 | List: [5, 6, 7]
Producer added 8 | List: [5, 6, 7, 8]
Consumer removed 5 | List: [6, 7, 8]
Producer added 9 | List: [6, 7, 8, 9]
Consumer removed 6 | List: [7, 8, 9]
Consumer removed 7 | List: [8, 9]
Consumer removed 8 | List: [9]
Consumer removed 9 | List: []
Final list: []


ques.5  Describe the methods and tools available in Python for safely sharing data between threads and
processes?

Sharing data safely between threads

Threads share the same memory space, so synchronization is essential.

1. threading.Lock / RLock

What: Mutual exclusion (mutex)
Use when: Protecting shared mutable data (lists, dicts, counters)

2. threading.Semaphore / BoundedSemaphore

What: Controls access to a limited resource
Use when: Allowing N threads to access something at once

3. threading.Condition

What: Thread coordination (wait/notify)
Use when: One thread must wait for a condition set by another

4. threading.Event

What: Signaling between threads
Use when: One thread needs to notify others to start/stop

5. queue.Queue

What: Thread-safe data exchange
Use when: Passing data between threads.

ques.6 Discuss why it’s crucial to handle exceptions in concurrent programs and the techniques available for
doing so?

1. Exceptions don’t propagate normally

In concurrent code:

.An exception in a thread does not crash the main program

.An exception in a process may terminate only that process

.The main program may continue in a corrupted or inconsistent state

2. Shared state can become corrupted

If a thread crashes while holding:

.a lock

.a file handle

.a network connection

  …other threads may:

.deadlock forever

.read invalid data

.fail unpredictably later

3. Resources may leak

Unhandled exceptions can leave behind:

.locked mutexes

.open files/sockets

.unclosed database connections

.orphaned processes

ques.7 Create a program that uses a thread pool to calculate the factorial of numbers from 1 to 10 concurrently.
Use concurrent.futures.ThreadPoolExecutor to manage the threads?

In [ ]:
from concurrent.futures import ThreadPoolExecutor, as_completed
import math

def factorial(n):
    return n, math.factorial(n)

def main():
    numbers = range(1, 11)

    with ThreadPoolExecutor(max_workers=4) as executor:
        futures = [executor.submit(factorial, n) for n in numbers]

        for future in as_completed(futures):
            n, result = future.result()
            print(f"Factorial of {n} is {result}")

if __name__ == "__main__":
    main()

Factorial of 5 is 120
Factorial of 4 is 24
Factorial of 9 is 362880
Factorial of 8 is 40320
Factorial of 3 is 6
Factorial of 1 is 1
Factorial of 2 is 2
Factorial of 6 is 720
Factorial of 7 is 5040
Factorial of 10 is 3628800


ques.8  Create a Python program that uses multiprocessing.Pool to compute the square of numbers from 1 to 10 in
parallel. Measure the time taken to perform this computation using a pool of different sizes (e.g., 2, 4, 8
processes).?

In [ ]:
import multiprocessing as mp
import time

def square(n):
    return n * n

def run_pool(pool_size, numbers):
    start_time = time.time()

    with mp.Pool(processes=pool_size) as pool:
        results = pool.map(square, numbers)

    end_time = time.time()
    elapsed_time = end_time - start_time

    print(f"Pool size: {pool_size}")
    print(f"Results: {results}")
    print(f"Time taken: {elapsed_time:.6f} seconds\n")

if __name__ == "__main__":
    numbers = list(range(1, 11))

    for pool_size in [2, 4, 8]:
        run_pool(pool_size, numbers)

Pool size: 2
Results: [1, 4, 9, 16, 25, 36, 49, 64, 81, 100]
Time taken: 0.029576 seconds

Pool size: 4
Results: [1, 4, 9, 16, 25, 36, 49, 64, 81, 100]
Time taken: 0.042275 seconds

Pool size: 8
Results: [1, 4, 9, 16, 25, 36, 49, 64, 81, 100]
Time taken: 0.059325 seconds

